# OpenAI agents playground notebook

Test and play with the following in this notebook:

* OpenAI API calling
* Creating agent tools
* Controling agent flow

### Setup

In [1]:
# imports
from openai import OpenAI
import json

In [ ]:
#API key
OPENAI_API_KEY = 

## OpenAI API Basics

### Our first LLM query

In [3]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [21]:
message = "What is the capital of Namibia"

messages = [
    {"role": "user", "content": message},
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

In [22]:
response

ChatCompletion(id='chatcmpl-B4PVtvxDkO4YryEpNWKSF6q7R9GLt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of Namibia is Windhoek.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1740391085, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_7fcd609668', usage=CompletionUsage(completion_tokens=9, prompt_tokens=13, total_tokens=22, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [ ]:
## Inspecting the response
print(json.dumps(response.model_dump(), indent=2))

In [24]:
print(response.choices[0].message.content)

The capital of Namibia is Windhoek.


 ### System prompt

In [26]:
SYSTEM_PROMPT = "Your are a helpful assistant knowledgeable about Namibia, please answer tourist queries in the most helpful manner."

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "What is the capital of Namibia"},
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

print(response.choices[0].message.content)

The capital of Namibia is Windhoek. It is located in the central part of the country and serves as the political, cultural, and economic center of Namibia. Windhoek is known for its blend of modern and colonial architecture, as well as its vibrant atmosphere and rich history. If you're planning to visit, you can explore various attractions including the Independence Memorial Museum, Christ Church, and the Namibia Craft Centre.


### Chat history

In [ ]:
messages = [
    {"role": "user", "content": "Give me a random number"},
]

response_1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

print(f"First answer: {response_1.choices[0].message.content}")

messages = [
    {"role": "user", "content": "Now add 10 to that number"},
]

response_2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

print(f"Second answer: {response_2.choices[0].message.content}")

First answer: Sure! Here's a random number: 47.
Second answer: Sure, but I need you to specify the number you want me to add 10 to. Please provide the number, and I'll be happy to help!


In [65]:
messages = [
    {"role": "user", "content": "Give me a random number"},
]

response_1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

messages.extend([
    {"role": "assistant", "content": response_1.choices[0].message.content},
    {"role": "user", "content": "Now add 10 to that number"},
])

response_2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

print(f"First response: {response_1.choices[0].message.content}")
print(f"Second response: {response_2.choices[0].message.content}")

First response: Sure! Here’s a random number: **27**.
Second response: Adding 10 to 27 gives you **37**.


### Hyperparameters

### max_completion_tokens

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What is the capital of Namibia"},
    ],
    max_completion_tokens=1
)

print(response.choices[0].message.content)

The


### timeout

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "What is the capital of Namibia?"}],
    timeout=0.1,
)

In [ ]:
# handling the timeout
from openai._exceptions import APITimeoutError

try:
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": "What is the capital of Namibia?"}],
        timeout=2  # Enforce a 2-second timeout
    )
    print(response)
except APITimeoutError:
    print("The request timed out. Try reducing the response complexity or increasing the timeout.")

### temperature

In [ ]:
for _ in range(3):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "Give me a random number. Output only the number."},
        ],
        temperature = 0.0,
    )
    print(response.choices[0].message.content)

42
42
42


In [ ]:
for _ in range(3):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "Give me a random number. Output only the number."},
        ],
        temperature = 1.0,
    )
    print(response.choices[0].message.content)

42
37
42
42
42


In [ ]:
for _ in range(3):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "Give me a random number. Output only the number."},
        ],
        temperature = 2.0,
    )
    print(response.choices[0].message.content)